In [106]:
import numpy as np
import cv2
import heapq

In [107]:
h = 250
w = 600
map = np.zeros((h,w,1), dtype=np.uint8)

In [108]:
text_set = set()
buffer_set = set()
border_set = set()
# boundary_map = map.copy()
# boundary_map = cv2.cvtColor(boundary_map,cv2.COLOR_BGR2GRAY)
# boundary_map = cv2.rectangle(boundary_map,(0,0),(w-1,h-1),255,1)
# border = np.where(boundary_map==255)
# for i in range(0,len(border[0])):
#     boundary.add((border[1][i],border[0][i]))

# del boundary_map
# del border

In [109]:
radius = 5
clearance = 5
buffer = radius+clearance

In [110]:
map_text = map.copy()
map_buffer = map.copy()
map_border = map.copy()

map_text = cv2.putText(map_text, "E N P M 6 6 1",(50,150), 0, 2, 255,5)
text_coord = np.where(map_text==255)
for i in range(0,len(text_coord[0])):
    xy = (text_coord[1][i],text_coord[0][i])
    text_set.add(xy)

for item in text_set:
    map_buffer = cv2.circle(map_buffer,item,buffer,255,-1)
buffer_coord = np.where(map_buffer==255)
for i in range(0,len(buffer_coord[0])):
    xy = (buffer_coord[1][i],buffer_coord[0][i])
    if xy not in text_set:
        buffer_set.add(xy)

map_border = cv2.rectangle(map_border,(0,0),(w-1,h-2),255,1)
border_coord = np.where(map_border==255)
for i in range(0,len(border_coord[0])):
    xy = (border_coord[1][i],border_coord[0][i])
    border_set.add(xy)

del map_text
del map_buffer
del map_border
del text_coord
del buffer_coord
del border_coord








In [111]:
display_map = np.zeros((h,w,3),dtype=np.uint8)
for item in text_set:
    display_map[item[1],item[0]] = (255,0,0)

for item in buffer_set:
    display_map[item[1],item[0]] = (0,255,0)

for item in border_set:
    display_map[item[1],item[0]] = (0,0,255)



In [112]:
# open list contains (total cost, (node_x,nodey))
# matrix contains (total cost, cost to come, (parent_x, parent_y))
open_list = []
ol_set = set()
closed_list = []
heapq.heapify(open_list)
heapq.heapify(closed_list)



start_x = 5
start_y = 5
start_t = 0
start_xy = (start_x,start_y)
start_xyt = (start_x,start_y,start_t)
end_x = 500
end_y = 240
end_t = 0
end_xy = (end_x,end_y)
end_xyt = (end_x,end_y,end_t)
step_size = 4




In [113]:
def distance(p1,p2):
    p1x,p1y,p1t = p1
    p2x,p2y,p1t = p2
    dist = np.sqrt(((p1x-p2x)**2)+((p1y-p2y)**2))
    dist = int(round(dist))
    return dist

def line_cross(p1,p2):
    p1x,p1y,p1t = p1
    p2x,p2y,p2t = p2
    p1xy = (p1x,p1y)
    p2xy = (p2x,p2y)
    l_map = np.zeros((h,w,1),dtype=np.uint8)
    l_map = cv2.line(l_map,p1xy,p2xy,(255),1)
    l_coords = np.where(l_map==255)
    for i in range(0,len(l_coords[0])):
        xy = (l_coords[1][i],l_coords[0][i])
        if xy in text_set:
            return True
        elif xy in buffer_set:
            return True
        elif xy in border_set:
            return True



In [114]:
# v1 = [(0,0,(0,0))]*12
# v2 = [v1]*w
# v = [v2]*h
# #v[h][w][t]

In [115]:
# v[25][25][10]

In [116]:
# v = np.zeros((h,w))


In [117]:
# open list contains (total cost, c2c, (parent xyt), (xyt))
v1 = [(0,0,(0,0,0),(0,0,0))]*w
v = [v1]*h

In [118]:
# open list contains (total cost, c2c, (parent xyt), (xyt))
#matrix only contains total costs
def move(node,angle):
    
    p_xyt = node[3]
    p_x = p_xyt[0]
    p_y = p_xyt[1]
    p_t = p_xyt[2]

    c_x = int(round(p_x+(step_size*np.cos(np.deg2rad(angle)))))
    c_y = int(round(p_y+(step_size*np.sin(np.deg2rad(angle)))))
    c_t = angle
    c_t_index = int(c_t/30)
    c_xyt = (c_x,c_y,c_t)

    if line_cross(p_xyt,c_xyt):
        return
    
    p_c2c = node[1]
    c_c2c = p_c2c+step_size
    c_c2g = distance(c_xyt,end_xyt)
    c_tot = c_c2c+c_c2g
    child_node = (c_tot,c_c2c,p_xyt,c_xyt)
    old_node = v[c_y][c_x]
    #print(old_node)
    if old_node == (0,0,(0,0,0),(0,0,0)):
        print("1")
        v[c_y][c_x] = child_node
        heapq.heappush(open_list,child_node)
        
    else:
        if old_node[0] < c_tot:
            print("2")
            
            return
        else:
            print("3")
            open_list.remove(old_node)
            v[c_y][c_x] = child_node
            heapq.heappush(open_list,child_node)
            return
        



In [119]:
start_ctg = distance(start_xyt,end_xyt)
first_OL = (start_ctg,0,start_xyt,start_xyt)
heapq.heappush(open_list,first_OL)
v[start_y][start_x] = first_OL

In [120]:
while True:
    parent_node = heapq.heappop(open_list)


    p_xyt = parent_node[3]
    dist_to_goal = distance(p_xyt,end_xyt)
    if dist_to_goal < 1.5:
        break

    heapq.heappush(closed_list,parent_node)

    for item in [-2,-1,0,1,2]:
        ang = p_xyt[2] + (30*item)
        ang = ang%360
        move(parent_node,ang)
    
    

1
1
1
3
3
1
1
3
1
2
2
2
1
3
3
3
3
2
2
2
2
1
2
2
2
2
2
1
3
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
3
2
1
3
2
2
2
2
1
3
3
2
2
1
2
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
2
2
2
2
1
3
2
2
1
3
2
2
2
1
3
3
2
2
1
3
3
2
2
2
2
2
2
1
2
2
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
3
2
1
3
2
2
2
2
1
3
3
2
2
1
2
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
3
2
3
3
2
2
2
2
1
3
3
2
2
1
2
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
3
2
2
1
3
3
2
3
3
2
2
2
2
1
2
2
2
2
2
1
3
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
2
2
1
3
2
2
2
2
1
3
3
2
2
2
1
3
2
1
1
3
2
2
2
1
3
3
2
2
2
1
3
3
3
2
2
2
2
2
2
1
2
2


IndexError: index out of range

In [ ]:
final_path = []
path_node = closed_list[-1][3]
while path_node != start_xyt:
    for item in closed_list:
        if item[3] == path_node:
            final_path.append(item)
            path_node = item[2]

final_path.reverse()

: 

In [ ]:
map_display = display_map.copy()
frame_skip = 10
i = 0
for item in closed_list:
    par_xy = (item[2][0],item[2][1])
    chi_xy = (item[3][0],item[3][1])
    cv2.line(map_display,par_xy,chi_xy,(255,255,255),1)
    cv2.circle(map_display,start_xy,2,(0,0,255),-1)
    cv2.circle(map_display,end_xy,2,(0,0,255),-1)
    frame = map_display.copy()
    if i%frame_skip == 0:
        cv2.imshow("animation",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    i+=1


for item in final_path:
    par_xy = (item[2][0],item[2][1])
    chi_xy = (item[3][0],item[3][1])
    cv2.line(map_display,par_xy,chi_xy,(0,0,255),1)
    cv2.circle(map_display,start_xy,2,(0,0,255),-1)
    cv2.circle(map_display,end_xy,2,(0,0,255),-1)
    frame = map_display.copy()
    
    cv2.imshow("animation",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

cv2.waitKey(0)
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/vboxuser/.local/lib/python3.10/site-packages/cv2/qt/plugins"
